실행 위치: Google Colab

# 1. vllm 라이브러리 설치

In [ ]:
!curl -LsSf https://astral.sh/uv/install.sh | sh

In [ ]:
!uv pip install --system vllm==0.11.0 torch==2.8.0 transformers==4.57.6

# 2. 모델 로드
 - 모델 로드 중 실패 발생시 제보 부탁드립니다. (Colab 기본 라이브러리 버전과 충돌 발생 가능성 있음)

In [ ]:
!vllm serve Qwen/Qwen3-1.7B > vllm_server.log 2>&1 &

In [ ]:
!cat vllm_server.log

INFO 01-04 08:01:03 [__init__.py:216] Automatically detected platform cuda.
2026-01-04 08:01:07.224517: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767513667.506547    2604 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767513667.579348    2604 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767513668.074784    2604 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767513668.074831    2604 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00

## Load상태 확인
 - Listen 포트 확인 (8000)
 - /v1/models API 호출
 - nvidia-smi

In [ ]:
!ss -tnlp

State  Recv-Q Send-Q Local Address:Port  Peer Address:PortProcess                                    
LISTEN 0      4096     172.28.0.12:6000       0.0.0.0:*    users:(("kernel_manager_",pid=11,fd=4))   
LISTEN 0      4096     172.28.0.12:37863      0.0.0.0:*    users:(("VLLM::EngineCor",pid=2878,fd=78))
LISTEN 0      128        127.0.0.1:3453       0.0.0.0:*    users:(("colab-fileshim.",pid=59,fd=3))   
LISTEN 0      2048         0.0.0.0:8000       0.0.0.0:*    users:(("vllm",pid=2604,fd=32))           
LISTEN 0      4096     172.28.0.12:40825      0.0.0.0:*    users:(("VLLM::EngineCor",pid=2878,fd=70))
LISTEN 0      100        127.0.0.1:33385      0.0.0.0:*    users:(("python3",pid=167,fd=28))         
LISTEN 0      4096     172.28.0.12:39021      0.0.0.0:*    users:(("VLLM::EngineCor",pid=2878,fd=86))
LISTEN 0      128      172.28.0.12:9000       0.0.0.0:*    users:(("jupyter-server",pid=108,fd=7))   
LISTEN 0      4096      127.0.0.11:39431      0.0.0.0:*                           

In [ ]:
!curl http://localhost:8000/v1/models

{"object":"list","data":[{"id":"Qwen/Qwen2.5-1.5B-Instruct","object":"model","created":1767514316,"owned_by":"vllm","root":"Qwen/Qwen2.5-1.5B-Instruct","parent":null,"max_model_len":32768,"permission":[{"id":"modelperm-aa2a8ebcf23c4766900c2bfca0f7b533","object":"model_permission","created":1767514316,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [ ]:
!nvidia-smi

Sun Jan  4 08:12:11 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   64C    P0             30W /   70W |   14312MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# 3. Online 추론 수행

## REST API 호출

In [ ]:
model = "Qwen/Qwen3-1.7B"
prompt = "Transformer model is "
max_tokens = 200
temperature = 0.0

In [ ]:
!curl http://localhost:8000/v1/completions \
    -H "Content-Type: application/json" \
    -d '{ \
        "model": "Qwen/Qwen3-1.7B", \
        "prompt": "Transformer model is a", \
        "max_tokens": 200, \
        "temperature": 0.0 \
    }'

{"id":"cmpl-98312b7b869f4fc9acbdfe02c3c3ad99","object":"text_completion","created":1767514348,"model":"Qwen/Qwen2.5-1.5B-Instruct","choices":[{"index":0,"text":" city of contrasts. The Golden Gate Bridge, with its distinctive red and white color, rises majestically over the bay. It’s an icon that has become synonymous with San Francisco, but there are also other attractions like the Haight-Ashbury district and Castro Street in the Castro neighborhood.\nThe Haight-Ashbury was a bohemian area during the 1960s hippie movement. Today, it has evolved into a popular tourist destination for people who want to experience a","logprobs":null,"finish_reason":"length","stop_reason":null,"token_ids":null,"prompt_logprobs":null,"prompt_token_ids":null}],"service_tier":null,"system_fingerprint":null,"usage":{"prompt_tokens":4,"total_tokens":104,"completion_tokens":100,"prompt_tokens_details":null},"kv_transfer_params":null}

## 파이썬 requests 호출

In [ ]:
import requests 

resp = requests.post(
    'http://localhost:8000/v1/completions',
    headers={"Content-Type: application/json"},
    json={
        "model": model, 
        "prompt": prompt, 
        "max_tokens": max_tokens,
        "temperature": temperature
    }
)

In [ ]:
print(resp)

## OpenAI API 호출
 - openAI 라이브러리는 기본 설치되어 있음


In [ ]:
from openai import OpenAI

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
completion = client.completions.create(model=model,
                                      prompt=prompt,
                                       max_tokens=max_tokens,
                                       temperature=temperature)

print("Completion result:", completion)

Completion result: Completion(id='cmpl-fe750a391fff4198ab7ba713e1c09860', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=None, text=" great city. It's full of exciting places and amazing experiences, but it also has its fair share of problems.\n\nOne such problem is the high cost of living in San Francisco. Housing costs are particularly high, making it difficult for many people to afford to live here. This can lead to increased stress levels, lower quality of life, and even homelessness. In addition, transportation costs are also expensive, with public transit being limited and parking fees being very high.\n\nAnother issue facing San Francisco is pollution.", stop_reason=None, token_ids=None, prompt_logprobs=None, prompt_token_ids=None)], created=1767514448, model='Qwen/Qwen2.5-1.5B-Instruct', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=100, prompt_tokens=4, total_tokens=104, completion_tokens_details=None, prompt_to

In [ ]:
!vllm serve --help

2025-06-18 03:50:26.001806: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750218626.290800    2245 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750218626.372052    2245 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-18 03:50:26.976202: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
INFO 06-18 03:50:32 [__init__.py:244] Automatically detected platform cuda.
usage: vllm serve [model_tag] [options]

Start th